In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="2"
os.environ["PYTHONPATH"]="/home/shariff/AVATAR"

In [2]:
!export PYTHONIOENCODING=utf-8
!export PYTHONPATH=/home/shariff/AVATAR

In [3]:
!python3 run_gen.py \
        --do_test \
        --model_type codet5 \
        --config_name Salesforce/codet5-base \
        --tokenizer_name Salesforce/codet5-base \
        --model_name_or_path Salesforce/codet5-base \
        --load_model_path /home/shariff/AVATAR/codet5/program/java2python/checkpoint-best-ppl/pytorch_model.bin \
        --task translate \
        --sub_task "java-python" \
        --data_dir /home/shariff/AVATAR/data \
        --cache_path /home/shariff/AVATAR/codet5/program/java2python \
        --output_dir /home/shariff/AVATAR/codet5/program/java2python \
        --res_dir /home/shariff/AVATAR/codet5/program/java2python \
        --max_source_length 510 \
        --max_target_length 510 \
        --beam_size 10 \
        --eval_batch_size 16 \
        2>&1 | tee /home/shariff/AVATAR/codet5/program/java2python/evaluation.log

In [4]:
from configs import add_args, set_seed, set_dist
from utils import get_filenames, get_elapse_time, load_and_cache_gen_data


In [5]:
class Args:
    do_test = True
    model_type = "codet5"
    config_name "Salesforce/codet5-base" 
    tokenizer_name = "Salesforce/codet5-base"
    model_name_or_path = "Salesforce/codet5-base"
    load_model_path = "/home/shariff/AVATAR/codet5/program/java2python/checkpoint-best-ppl/pytorch_model.bin"
    task = "translate"
    sub_task = "java-python"
    data_dir = "/home/shariff/AVATAR/data"
    cache_path = "/home/shariff/AVATAR/codet5/program/java2python"
    output_dir = "/home/shariff/AVATAR/codet5/program/java2python"
    res_dir = "/home/shariff/AVATAR/codet5/program/java2python"
    max_source_length = 510 \
    max_target_length = 510 \
    beam_size = 10
    eval_batch_size = 8
    
args = Args()

In [6]:
set_dist(args)
set_seed(args)
config, model, tokenizer = build_or_load_gen_model(args)
model.to(args.device)
pool = multiprocessing.Pool(args.cpu_cont)
args.train_filename, args.dev_filename, args.test_filename = get_filenames(args.data_dir, args.task, args.sub_task)
fa = open(os.path.join(args.output_dir, 'summary.log'), 'a+')
